In [3]:
import os
from dotenv import load_dotenv
from pprint import pprint

# LangGraph + LangChain
from langgraph.graph import StateGraph, MessagesState, END
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_openai import ChatOpenAI

# Tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

# Memory
from langgraph.checkpoint.memory import MemorySaver

# Messages
from langchain_core.messages import HumanMessage

# LangSmith Tracer
from langchain.callbacks.tracers.langchain import LangChainTracer

# ------------------- ENV LOAD -------------------
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "LangGraph-ReAct-BindTools"

# ------------------- TRACER -------------------
tracer = LangChainTracer()

# ------------------- TOOLS -------------------
# Tool 1: Tavily Search
tavily_tool = TavilySearchResults(max_results=3)

# Tool 2: Wikipedia
wiki_wrapper = WikipediaAPIWrapper()
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

# Tool 3: Arxiv (Research Papers)
arxiv_wrapper = ArxivAPIWrapper()
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

# ------------------- LLM -------------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Bind tools to LLM
llm_with_tools = llm.bind_tools([tavily_tool, wiki_tool, arxiv_tool])

# ------------------- AGENT NODE -------------------
def agent_node(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# ------------------- TOOL NODE -------------------
tool_node = ToolNode([tavily_tool, wiki_tool, arxiv_tool])

# ------------------- MEMORY -------------------
memory = MemorySaver()

# ------------------- GRAPH -------------------
graph = StateGraph(MessagesState)

# Nodes
graph.add_node("agent", agent_node)
graph.add_node("tools", tool_node)

# Conditional edge: agent → tools or END
graph.add_conditional_edges(
    "agent",
    tools_condition,
    {
        "tools": "tools",
        END: END
    }
)

# Unconditional edge: tools → agent
graph.add_edge("tools", "agent")

# Entry point
graph.set_entry_point("agent")

# Compile graph with memory
app = graph.compile(checkpointer=memory)


def run_tests():
    """Run example queries to test the agent."""
    thread_id = "bilal-thread-006"

    # 1st call
    result1 = app.invoke(
        {"messages": [HumanMessage(content="Find AI agent trends in 2025 using web search")]},
        config={"thread_id": thread_id, "callbacks": [tracer]}
    )
    print("\n--- First Answer ---")
    pprint(result1["messages"][-1].content)

    # 2nd call (memory retrieval)
    result2 = app.invoke(
        {"messages": [HumanMessage(content="Summarize in 3 bullet points")]},
        config={"thread_id": thread_id, "callbacks": [tracer]}
    )
    print("\n--- Second Answer (Memory Retrieved) ---")
    pprint(result2["messages"][-1].content)

    # 3rd call (Wikipedia example)
    result3 = app.invoke(
        {"messages": [HumanMessage(content="What is the population of Japan according to Wikipedia?")]},
        config={"thread_id": thread_id, "callbacks": [tracer]}
    )
    print("\n--- Third Answer (Wikipedia Tool) ---")
    pprint(result3["messages"][-1].content)

    # 4th call (Arxiv example)
    result4 = app.invoke(
        {"messages": [HumanMessage(content="Find recent research papers on AI agents")]},
        config={"thread_id": thread_id, "callbacks": [tracer]}
    )
    print("\n--- Fourth Answer (Arxiv Tool) ---")
    pprint(result4["messages"][-1].content)

    # Display graph
    print("\n--- Graph Structure (Mermaid) ---")
    print(app.get_graph().draw_mermaid())


if __name__ == "__main__":
    run_tests()



--- First Answer ---
('Here are some key trends regarding AI agents in 2025 based on recent '
 'findings:\n'
 '\n'
 '1. **Multi-Agent Systems**: There is a significant shift towards multi-agent '
 'systems where specialized agents collaborate to solve problems. This '
 '"orchestra" approach allows different agents to handle tasks they are best '
 'suited for, enhancing overall efficiency and capability.\n'
 '\n'
 '2. **Expanded Toolsets**: AI agents in 2025 have access to a broader range '
 'of tools, including web browsing, code execution, and integration with '
 'enterprise applications. This is facilitated by standard interfaces like '
 'APIs, allowing agents to tackle more complex, multi-step projects.\n'
 '\n'
 '3. **Productivity Gains**: Implementing AI agents has led to notable '
 'productivity increases, with reports indicating a 7.8% rise in overall '
 'productivity and a 30% reduction in time spent on repetitive tasks. Customer '
 'satisfaction has also improved, with some c